In [1]:
import sys

sys.path.insert(0, '../auxiliary_scripts/')

In [2]:
import numpy as np
import vtktools
import torch
import time 

In [3]:
def get_edge_list_for_node(input, vtu_graph):
    
    node_neighbours = vtu_graph.GetPointPoints(input)
    node_edge_list = []
    
    for node in node_neighbours:
        if node > input:  # change to >= to include the same node
            node_edge_list.append([input, node])
            
    return node_edge_list

def get_complete_edge_list(vtu_object, n_points):

    complete_edge_list = []

    for point in range(n_points):  #
        complete_edge_list = complete_edge_list + get_edge_list_for_node(point, vtu_object)
        
    complete_edge_list = np.array(complete_edge_list)
    # complete_edge_list = torch.tensor(complete_edge_list).t()
        
    return complete_edge_list

In [4]:
location = '../../Datasets/WaveSuite_VTK/sateNo4_1_' 
start_index = 40
end_index = 100  # CHANGE back to 100

vtu_file_locations = [ location + str(i) + '.vtu' for i in range(start_index, end_index)]
features_list = ['nut'] 

In [5]:
#First object
first_object = vtktools.vtu(vtu_file_locations[0]) 

In [6]:
#This two parameters are the same for every graph, so they would be calculated just once instead of in every iteration
n_points = first_object.GetField(features_list[0]).shape[0]
n_points 

851101

In [7]:
start = time.time()

complete_edge_list = get_complete_edge_list(first_object, n_points)

end = time.time()

print(end-start)

70199.95070791245


In [8]:
filename = 'edges_lists/wave_complete_edge_list'

In [9]:
np.save(filename+'.npy',complete_edge_list)

In [10]:
np.load(filename+'.npy')

array([[     0,      1],
       [     0,   2721],
       [     0,   2720],
       ...,
       [824531, 824532],
       [824532, 850703],
       [824532, 850866]])

In [11]:
start = time.time()

complete_edge_list = torch.tensor(complete_edge_list).t()

end = time.time()

print(end-start)

0.015617132186889648


In [12]:
torch.save(complete_edge_list, filename+'.pt')

In [13]:
torch.load(filename+'.pt')

tensor([[     0,      0,      0,  ..., 824531, 824532, 824532],
        [     1,   2721,   2720,  ..., 824532, 850703, 850866]],
       dtype=torch.int32)